In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


**Predicting Student admissions based on their GRE scores, GPA scores and Class rank using Neural Networks**

In [32]:
import pandas as pd
student_data = pd.read_csv("student_data.csv")

In [33]:
student_data[:10]

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
5,1,760,3.00,2
6,1,560,2.98,1
7,0,400,3.08,2
8,1,540,3.39,3
9,0,700,3.92,2


In [34]:
student_data = pd.concat([student_data,pd.get_dummies(student_data['rank'],prefix='rank')],axis=1)

In [35]:
student_data.drop(['rank'],axis=1,inplace=True)

In [36]:
student_data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [37]:
#Scale the features as these are different magnitudes. I am doing simple scaling here, just dividing values by maximum value.
print(student_data['gre'].max())
print(student_data['gpa'].max())

800
4.0


In [38]:
#Copying the data
processed_data = student_data[:]
processed_data['gre'] = processed_data['gre']/800
processed_data['gpa'] = processed_data['gpa']/4.0
processed_data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,0.475,0.9025,0,0,1,0
1,1,0.825,0.9175,0,0,1,0
2,1,1.000,1.0000,1,0,0,0
3,1,0.800,0.7975,0,0,0,1
4,0,0.650,0.7325,0,0,0,1


In [39]:
sample = np.random.choice(processed_data.index,int(len(processed_data)*0.9),replace=False)
train, test = processed_data.iloc[sample], processed_data.drop(sample)
print("Number of training samples is ",len(train))
print("Number of test samples is ",len(test))
print(train.head())
print(test.head())

Number of training samples is  360
Number of test samples is  40
     admit    gre     gpa  rank_1  rank_2  rank_3  rank_4
175      1  0.775  0.8425       0       1       0       0
263      1  0.775  0.9875       0       0       1       0
198      0  0.750  0.9075       0       0       1       0
298      0  0.675  0.8750       0       1       0       0
390      1  1.000  0.7625       0       1       0       0
    admit    gre     gpa  rank_1  rank_2  rank_3  rank_4
0       0  0.475  0.9025       0       0       1       0
5       1  0.950  0.7500       0       1       0       0
7       0  0.500  0.7700       0       1       0       0
40      0  0.700  0.6050       0       1       0       0
44      0  0.875  0.7350       0       1       0       0


In [40]:
features = train.drop('admit',axis=1)
targets = train['admit']
features_test = test.drop('admit',axis=1)
targets_test = test['admit']
print(features.head())
print(targets.head())

       gre     gpa  rank_1  rank_2  rank_3  rank_4
175  0.775  0.8425       0       1       0       0
263  0.775  0.9875       0       0       1       0
198  0.750  0.9075       0       0       1       0
298  0.675  0.8750       0       1       0       0
390  1.000  0.7625       0       1       0       0
175    1
263    1
198    0
298    0
390    1
Name: admit, dtype: int64


In [41]:
len(processed_data.index)

400

Training the neural network using Cross entropy as loss function

In [42]:
epochs = 1000
learnrate = 0.5

def sigmoid(x):
    return 1/(1+np.exp(-x))

def error_formula(y,output):
    return -y*np.log(output)-(1-y)*np.log(1-output)

def error_derivative(y, output):
    return -(y-output)

def train_nn(fetaures, targets, epochs, learnrate):
    #Use same seed to make debugging easier
    np.random.seed(101)
    n_records, n_features = features.shape
    last_loss = None
    #Initialize weights
    weights = np.random.normal(scale=1/n_features**0.5,size=n_features)
    #print(weights.shape)
    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x,y in zip(features.values,targets):
            #print("features shape",features.shape)
            #print("x shape {} weights shape{} np.dot {}".format(x.shape,weights.shape,np.dot(x,weights)))
            layer1_output = sigmoid(np.dot(x,weights))
            #print("output:",layer1_output)
            errors = error_formula(y,layer1_output)
            #print(error_derivative(y, layer1_output).shape)
            del_w += error_derivative(y, layer1_output)*x
            #print("del_W ",del_w)
        weights -= learnrate * del_w / n_records
        #print("weights ",weights)
        out = sigmoid(np.dot(features,weights))
        #print("out.shape ",out.shape)
        loss = np.mean(-targets*np.log(out)-(1-targets)*np.log(1-out))
        print("loss:",loss)
    return weights

In [43]:
weights = train_nn(features, targets, epochs, learnrate)

loss: 0.9233300486138347
loss: 0.8214231824812436
loss: 0.7514468114638546
loss: 0.7047914783336738
loss: 0.6741052371781558
loss: 0.6539555080811288
loss: 0.6406390339783299
loss: 0.6317335787494737
loss: 0.6256829667698518
loss: 0.6214921402539695
loss: 0.6185233536339171
loss: 0.6163654864956146
loss: 0.6147515656555629
loss: 0.613506868241363
loss: 0.6125161509287999
loss: 0.6117028180242099
loss: 0.6110155736721908
loss: 0.6104197996459004
loss: 0.6098919421545621
loss: 0.6094158312240604
loss: 0.6089802515989484
loss: 0.6085773303362242
loss: 0.6082014610212537
loss: 0.6078485827403195
loss: 0.6075156948391188
loss: 0.6072005291272107
loss: 0.6069013276442826
loss: 0.6066166914525816
loss: 0.6063454773675028
loss: 0.606086727133558
loss: 0.6058396186162368
loss: 0.605603431969322
loss: 0.6053775260136706
loss: 0.6051613215970819
loss: 0.6049542897407052
loss: 0.6047559430785318
loss: 0.6045658295719838
loss: 0.604383527804515
loss: 0.6042086433807723
loss: 0.6040408061044337
loss

loss: 0.5961187897310082
loss: 0.5961059504293027
loss: 0.5960931244764737
loss: 0.5960803118450321
loss: 0.5960675125076931
loss: 0.5960547264373723
loss: 0.5960419536071841
loss: 0.5960291939904312
loss: 0.5960164475606049
loss: 0.5960037142913758
loss: 0.5959909941565965
loss: 0.5959782871302873
loss: 0.5959655931866432
loss: 0.5959529123000219
loss: 0.5959402444449428
loss: 0.5959275895960818
loss: 0.595914947728271
loss: 0.5959023188164922
loss: 0.5958897028358733
loss: 0.5958770997616885
loss: 0.5958645095693487
loss: 0.5958519322344072
loss: 0.5958393677325481
loss: 0.5958268160395891
loss: 0.5958142771314761
loss: 0.5958017509842808
loss: 0.5957892375741991
loss: 0.5957767368775468
loss: 0.595764248870759
loss: 0.5957517735303863
loss: 0.5957393108330943
loss: 0.5957268607556568
loss: 0.5957144232749614
loss: 0.5957019983679975
loss: 0.595689586011864
loss: 0.5956771861837601
loss: 0.5956647988609881
loss: 0.5956524240209476
loss: 0.5956400616411341
loss: 0.5956277116991447
los

loss: 0.5924577039960847
loss: 0.5924482636842844
loss: 0.5924388311663907
loss: 0.5924294064311169
loss: 0.5924199894671982
loss: 0.5924105802633942
loss: 0.5924011788084851
loss: 0.5923917850912737
loss: 0.5923823991005862
loss: 0.5923730208252705
loss: 0.5923636502541983
loss: 0.5923542873762603
loss: 0.5923449321803704
loss: 0.5923355846554681
loss: 0.5923262447905097
loss: 0.5923169125744769
loss: 0.5923075879963721
loss: 0.5922982710452211
loss: 0.5922889617100692
loss: 0.5922796599799853
loss: 0.592270365844059
loss: 0.5922610792914029
loss: 0.5922518003111492
loss: 0.5922425288924548
loss: 0.5922332650244952
loss: 0.5922240086964689
loss: 0.5922147598975958
loss: 0.5922055186171165
loss: 0.5921962848442944
loss: 0.5921870585684117
loss: 0.5921778397787756
loss: 0.5921686284647107
loss: 0.5921594246155657
loss: 0.5921502282207093
loss: 0.5921410392695304
loss: 0.5921318577514408
loss: 0.5921226836558711
loss: 0.5921135169722755
loss: 0.5921043576901271
loss: 0.5920952057989204
l

loss: 0.5896933081979987
loss: 0.5896859500151412
loss: 0.5896785968646033
loss: 0.5896712487404105


Training the neural network using Mean Squared Error as loss function

In [44]:
epochs = 1000
learnrate = 0.5

def sigmoid(x):
    return 1/(1+np.exp(-x))

def error_formula(y,output):
    return (y - output) ** 2

def error_derivative(y, output):
    return (y-output)*output*(1-output)

def train_nn(fetaures, targets, epochs, learnrate):
    #Use same seed to make debugging easier
    np.random.seed(101)
    n_records, n_features = features.shape
    last_loss = None
    #Initialize weights
    weights = np.random.normal(scale=1/n_features**0.5,size=n_features)
    print(weights.shape)
    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x,y in zip(features.values,targets):
            #print("features shape",features.shape)
            #print("x shape {} weights shape{} np.dot {}".format(x.shape,weights.shape,np.dot(x,weights)))
            layer1_output = sigmoid(np.dot(x,weights))
            #print("output:",layer1_output)
            errors = error_formula(y,layer1_output)
            #print(error_derivative(y, layer1_output).shape)
            del_w += error_derivative(y, layer1_output)*x
            #print("del_W ",del_w)
        weights += learnrate * del_w / n_records
        #print("weights ",weights)
        out = sigmoid(np.dot(features,weights))
        #print("out.shape ",out.shape)
        loss = np.mean((targets-out)**2)
        print("loss:",loss)
    return weights

In [45]:
weights = train_nn(features, targets, epochs, learnrate)

(6,)
loss: 0.4014798945892641
loss: 0.39177042995273326
loss: 0.3819833655692279
loss: 0.3721758396872349
loss: 0.3624088313288105
loss: 0.35274546092320574
loss: 0.34324902073110813
loss: 0.33398086316469505
loss: 0.32499830367827925
loss: 0.3163527040368705
loss: 0.3080878879986713
loss: 0.3002390060742777
loss: 0.2928319150930898
loss: 0.2858830811071436
loss: 0.2793999607724778
loss: 0.27338177508775974
loss: 0.267820565030246
loss: 0.26270241194766486
loss: 0.25800871388119645
loss: 0.25371742762633837
loss: 0.24980421008480674
loss: 0.24624341678466813
loss: 0.24300893712114735
loss: 0.24007486312011647
loss: 0.23741600081166603
loss: 0.23500824094915687
loss: 0.2328288096262498
loss: 0.2308564203057241
loss: 0.22907134781539457
loss: 0.2274554427740156
loss: 0.22599210228052094
loss: 0.224666209947406
loss: 0.22346405574395173
loss: 0.22237324378078402
loss: 0.22138259417507603
loss: 0.22048204349347766
loss: 0.2196625469543858
loss: 0.21891598454336522
loss: 0.21823507241090276

loss: 0.20595688237710283
loss: 0.20595224904693257
loss: 0.20594764447711522
loss: 0.20594306838935528
loss: 0.2059385205083008
loss: 0.2059340005615069
loss: 0.20592950827940573
loss: 0.20592504339527076
loss: 0.2059206056451863
loss: 0.20591619476801395
loss: 0.2059118105053622
loss: 0.20590745260155421
loss: 0.20590312080359674
loss: 0.20589881486115003
loss: 0.2058945345264987
loss: 0.20589027955451952
loss: 0.20588604970265356
loss: 0.2058818447308783
loss: 0.20587766440167588
loss: 0.20587350848000746
loss: 0.20586937673328368
loss: 0.20586526893133847
loss: 0.20586118484640006
loss: 0.2058571242530647
loss: 0.20585308692827126
loss: 0.20584907265127173
loss: 0.20584508120360967
loss: 0.20584111236909092
loss: 0.20583716593375925
loss: 0.20583324168587183
loss: 0.2058293394158744
loss: 0.2058254589163764
loss: 0.20582159998212704
loss: 0.20581776240999158
loss: 0.20581394599892705
loss: 0.20581015054996057
loss: 0.20580637586616457
loss: 0.20580262175263483
loss: 0.2057988880164

loss: 0.20513674075676494
loss: 0.20513514129494798
loss: 0.20513354370366663
loss: 0.20513194797059353
loss: 0.20513035408350536
loss: 0.2051287620302828
loss: 0.20512717179890702
loss: 0.20512558337746267
loss: 0.205123996754133
loss: 0.20512241191720196
loss: 0.20512082885505092
loss: 0.2051192475561599
loss: 0.20511766800910544
loss: 0.20511609020255964
loss: 0.2051145141252901
loss: 0.205112939766158
loss: 0.20511136711411918
loss: 0.20510979615822061
loss: 0.20510822688760152
loss: 0.20510665929149216
loss: 0.2051050933592125
loss: 0.2051035290801721
loss: 0.2051019664438681
loss: 0.20510040543988667
loss: 0.20509884605789935
loss: 0.20509728828766519
loss: 0.2050957321190272
loss: 0.20509417754191414
loss: 0.20509262454633723
loss: 0.20509107312239203
loss: 0.20508952326025548
loss: 0.2050879749501866
loss: 0.20508642818252515
loss: 0.2050848829476905
loss: 0.20508333923618208
loss: 0.20508179703857737
loss: 0.20508025634553279
loss: 0.2050787171477808
loss: 0.20507717943613127


loss: 0.20468022400252947
loss: 0.2046789110063883
loss: 0.20467759845681702
loss: 0.20467628635273036
loss: 0.2046749746930491
loss: 0.20467366347670185
loss: 0.20467235270262374
loss: 0.20467104236975678
loss: 0.20466973247705
loss: 0.20466842302345883
loss: 0.20466711400794652
loss: 0.20466580542948154
loss: 0.20466449728704003
loss: 0.20466318957960433
loss: 0.2046618823061639
loss: 0.20466057546571395
loss: 0.20465926905725632
loss: 0.2046579630797996
loss: 0.2046566575323584


In [50]:
#training accuracy
train_out = sigmoid(np.dot(features.values,weights))
pred = train_out > 0.5
print("training accuracy:",np.mean(targets == pred))

training accuracy: 0.6972222222222222


In [51]:
test_out = sigmoid(np.dot(features_test,weights))
pred = test_out > 0.5
print("testing accuracy:",np.mean(targets_test == pred))

testing accuracy: 0.675
